In [47]:
import pandas as pd
from sklearn.cross_validation import train_test_split
from matplotlib import pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
#import Recommenders as Recommenders
from surprise import Reader, Dataset, SVD, evaluate

In [48]:
dt = pd.read_csv('Custmer_Fuel_Data.csv', low_memory=False)
dt.head()

,ID,Vehicle_No,Car_Type,Volume_Fuel,Car_Age,Type_Fuel,Gender,Age,NFR
0,1,KA-03 HJ 9082,ETIOS,10,NEW,V-Power 98,Male,1-30,0
1,2,KA-03 HJ 9083,Alto 800,5,AVG,Premium Unleaded 98,Female,31-60,0
2,3,KA-03 HJ 9084,Hyundai Grand i10,20,OLD,Unleaded 95,Male,1-30,0
3,4,KA-03 HJ 9085,ETIOS,10,NEW,Unleaded 91,Male,1-30,0
4,5,KA-03 HJ 9086,Alto 800,4,AVG,Unleaded E10,Male,1-30,0


In [49]:
dt.dropna(inplace=True)   # Remove missing values

In [50]:
dt_filtered = dt[(dt.Car_Type == 'ETIOS') & (dt.Car_Age == 'NEW') & (dt.Gender == 'Male') & (dt.Age == '1-30') & (dt.NFR == 0)] 
dt_filtered

,ID,Vehicle_No,Car_Type,Volume_Fuel,Car_Age,Type_Fuel,Gender,Age,NFR
0,1,KA-03 HJ 9082,ETIOS,10,NEW,V-Power 98,Male,1-30,0
3,4,KA-03 HJ 9085,ETIOS,10,NEW,Unleaded 91,Male,1-30,0
9,10,KA-03 HJ 9082,ETIOS,10,NEW,V-Power 98,Male,1-30,0
12,13,KA-03 HJ 9085,ETIOS,10,NEW,Unleaded 91,Male,1-30,0
18,19,KA-03 HJ 9082,ETIOS,20,NEW,Unleaded E10,Male,1-30,0
28,29,KA-03 HJ 9085,ETIOS,10,NEW,Unleaded 91,Male,1-30,0
68,69,KA-03 HJ 9082,ETIOS,9,NEW,Unleaded 91,Male,1-30,0
69,70,KA-03 HJ 9082,ETIOS,11,NEW,Unleaded 91,Male,1-30,0
72,73,KA-03 HJ 9082,ETIOS,17,NEW,V-Power 98,Male,1-30,0
73,74,KA-03 HJ 9082,ETIOS,19,NEW,Unleaded 91,Male,1-30,0


In [51]:
#x_train, x_test = train_test_split(dt,test_size=0.05)

dt_filtered['Gender'].replace({'Female':0,'Male':1},inplace=True)
dt_filtered['Car_Age'].replace({'OLD':0,'AVG':1, 'NEW':2},inplace=True)

dt_filtered['Age'].replace({
   '1-30': 0,
   '31-60': 1,
   '61-90': 2
},inplace=True)

dt_filtered['Type_Fuel'].replace({
    'V-Power 98': 1,
    'Premium Unleaded 98': 2,
    'Unleaded 95': 3,
    'Unleaded 91': 4,
    'Unleaded E10': 5,
    'Unleaded 91 Low Aromatic': 6,
    'V-Power Diesel': 7,
    'Diesel Extra': 8,
    'Diesel': 9
},inplace=True)

dt_filtered['Car_Type'].replace({
    'ETIOS':0,
    'Alto 800':1,
    'Hyundai Grand i10':2,
    'ETIOS':3,
    'Ford Figo':4,
    'Hyundai Accent':5,
    'Honda city':6
},inplace=True)

dt_filtered.drop(columns = ['ID', 'Vehicle_No'])
data = pd.DataFrame(dt_filtered)
data = data.drop(['ID', 'Vehicle_No'], axis=1)
data.head()

,Car_Type,Volume_Fuel,Car_Age,Type_Fuel,Gender,Age,NFR
0,3,10,2,1,1,0,0
3,3,10,2,4,1,0,0
9,3,10,2,1,1,0,0
12,3,10,2,4,1,0,0
18,3,20,2,5,1,0,0


In [25]:
from sklearn.metrics.pairwise import euclidean_distances
euclidean_distances(data, data)

array([[ 0.        ,  8.1240384 ,  6.08276253, ...,  6.08276253,
         5.38516481,  3.87298335],
       [ 8.1240384 ,  0.        ,  8.30662386, ...,  5.91607978,
         7.28010989,  7.68114575],
       [ 6.08276253,  8.30662386,  0.        , ..., 10.09950494,
         2.        ,  8.36660027],
       ...,
       [ 6.08276253,  5.91607978, 10.09950494, ...,  0.        ,
         8.94427191,  3.74165739],
       [ 5.38516481,  7.28010989,  2.        , ...,  8.94427191,
         0.        ,  7.61577311],
       [ 3.87298335,  7.68114575,  8.36660027, ...,  3.74165739,
         7.61577311,  0.        ]])

In [88]:
# The columns that we will be making predictions with.
x_columns = ['Car_Type', 'Volume_Fuel', 'Car_Age', 'Gender', 'Age', 'NFR']
# The column that we want to predict.
y_column = ["Type_Fuel"]

from sklearn.neighbors import KNeighborsRegressor
# Create the knn model.
# Look at the five closest neighbors.
knn = KNeighborsRegressor(n_neighbors = 1, weights='uniform', p=2, metric='euclidean')
# Fit the model on the training data.
knn.fit(data[x_columns], data[y_column])
# Make point predictions on the test set using the fit model.


KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='euclidean',
          metric_params=None, n_jobs=1, n_neighbors=1, p=2,
          weights='uniform')

In [89]:
testdata = {'Car_Type': ['ETIOS'], 'Volume_Fuel' : [5], 'Car_Age' : ['NEW'], 'Gender' : ['Male'], 'Age' : ['1-30'], 'NFR' : [0] }
testdata = pd.DataFrame(data=testdata, columns=['Car_Type', 'Volume_Fuel', 'Car_Age', 'Gender', 'Age', 'NFR'])

testdata['Gender'].replace({'Female':0,'Male':1},inplace=True)
testdata['Car_Age'].replace({'OLD':0,'AVG':1, 'NEW':2},inplace=True)

testdata['Age'].replace({
   '1-30': 0,
   '31-60': 1,
   '61-90': 2
},inplace=True)

testdata['Car_Type'].replace({
    'ETIOS':0,
    'Alto 800':1,
    'Hyundai Grand i10':2,
    'ETIOS':3,
    'Ford Figo':4,
    'Hyundai Accent':5,
    'Honda city':6
},inplace=True)

testdata = pd.DataFrame(testdata)
testdata.head()

,Car_Type,Volume_Fuel,Car_Age,Gender,Age,NFR
0,3,5,2,1,0,0


In [90]:
predictions = knn.predict(testdata[x_columns])

In [91]:
predictions
#predictions[0][0]

array([[4.]])

In [92]:
def f(x):
    return {
        1 : 'V-Power 98',
        2 : 'Premium Unleaded 98',
        3 : 'Unleaded 95',
        4 : 'Unleaded 91',
        5 : 'Unleaded E10',
        6 : 'Unleaded 91 Low Aromatic',
        7 : 'V-Power Diesel',
        8 : 'Diesel Extra',
        9 : 'Diesel'
    }[x]

In [93]:
recommendation = f(int(predictions[0][0]))
recommendation

'Unleaded 91'